In [5]:
from __future__ import print_function
import os,sys
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import torch
from larvoxelclass_dataset import larvoxelClassDataset
from detectoroutline import DetectorOutline

In [3]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
ssnet_color_array = np.array( ((0,0,0),      # bg
                               (255,0,0),    # electron
                               (0,255,0),    # gamma
                               (0,0,255),    # muon
                               (255,0,255),  # pion
                               (0,255,255),  # proton
                               (0,0,0)),     # other
                               dtype=np.float64 ) 

larcvpid2class = {0:0,#unknown -> bg
                  1:0,#cosmic -> bg
                  2:0,#bnb -> bg
                  3:1,#eminus->electron
                  4:2,#gamma->gamma
                  5:2,#pizero->gamma
                  6:3,#muon->muon
                  7:4,#kaon->other
                  8:4,#pion->pion
                  9:5,#proton->proton
                 }

In [6]:
# DATA LOADER
batch_size = 1
test = larvoxelClassDataset( filelist=["testdata/testdata.root"],
                             random_access=False, 
                            load_truth=True)
print("NENTRIES: ",len(test))
loader = torch.utils.data.DataLoader(test,
                                     batch_size=batch_size,
                                     collate_fn=larvoxelClassDataset.collate_fn)


NENTRIES:  468


In [100]:
# Get entry data
batch = next(iter(loader))
nvoxels = batch[0]["coord"].shape[0]
# We need to retrieved the 3d positions
pos3d = batch[0]["coord"].astype(np.float64)*0.3
pos3d[:,1] -= 117.0 
print(pos3d.shape)
print("PID: ",batch[0]["pid"])

(4183, 3)
PID:  [0]


In [101]:
# Plot Particle
print("PID: ",batch[0]["pid"])
no_ghost_points = True
pos = pos3d
#labelcol = (np.clip( batch[0]["feat"], 0, 3.0 )/3.0)[:,2]
labelcol = np.mean( np.clip( batch[0]["feat"], 0, 3.0 )/3.0,axis=1 )
ave3d = np.mean(pos,axis=0,keepdims=True).squeeze()
ave3d[0] += 125.0
ave3d[2] -= 1036
ave3d[0] /= 256
ave3d[1] /= 117.0
ave3d[2] /= 1036

print(ave3d)
print(labelcol.shape)

particle_plot = {
    "type":"scatter3d",
    "x":pos[:,0],
    "y":pos[:,1],
    "z":pos[:,2],
        "mode":"markers",
        "name":"particle",
        "marker":{"color":labelcol,"size":1,"colorscale":"Viridis"}
    }

detlines = detdata.getlines(color=(10,10,10))

# DATA
particle_plot_data = [particle_plot] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='SINGLE PARTICLE VOXELS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": ave3d[0]+0.1, "y": ave3d[1], "z": ave3d[2]},
                   "center":dict(x=ave3d[0], y=ave3d[1], z=ave3d[2]),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=particle_plot_data, layout=layout)
fig.show()

PID:  [0]
[ 1.00001018 -0.75224566 -0.43744701]
(4183,)


In [ ]:
# NOTES
# Probably need to do a DB-scan and remove detached small, low-charge sub-clusters. 
# I suspect these can be used to build a unique code to overtrain on.
# Plot the space-charge corrected track to verify charge cluster